In [2]:
from optuna import Trial
from typing import Dict, Union, Any
from setfit import TrainingArguments
from setfit import SetFitModel

/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")
    params = {
        "head_params": {
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    return SetFitModel.from_pretrained("KBLab/sentence-bert-swedish-cased", **params)


In [4]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "body_learning_rate": trial.suggest_float("body_learning_rate", 1e-6, 1e-3, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 5),
        "batch_size": trial.suggest_categorical("batch_size", [8, 16]),
        "seed": trial.suggest_int("seed", 1, 40),
        "max_iter": trial.suggest_int("max_iter", 50, 300),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

The input is a csv file with three columns:
1) Requirement ID
2) Requirement text
3) Boolean label, which indicates whether all judges agreed in the classification

In [21]:
from datasets import load_dataset, DatasetDict
dataset = load_dataset("csv", data_files="../data/target_pilot_1_2_sv.csv")
dataset = dataset.class_encode_column("label")
dataset['train'].features

{'ID': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['False', 'True'], id=None)}

In [6]:
train_testvalid = dataset['train'].train_test_split(test_size=0.2, stratify_by_column='label')
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, stratify_by_column='label')
ttv_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [7]:
ttv_dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 57
    })
    test: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 8
    })
    valid: Dataset({
        features: ['ID', 'text', 'label'],
        num_rows: 7
    })
})

In [16]:
ttv_dataset["train"][0]

{'ID': 'K2770',
 'text': 'Överkant på bomarm i avstängningsanordningar vid öppningsbar bro, vägoperativ miljö, ska vara 0,9 – 1,1 m över körbanan i stängt läge.',
 'label': 1}

In [8]:
from setfit import Trainer

In [9]:
trainer = Trainer(
    model_init=model_init,
    train_dataset=ttv_dataset["train"],
    eval_dataset=ttv_dataset["valid"]
)

/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lin

In [10]:
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=100)
print(best_run)

[I 2024-11-27 17:14:46,258] A new study created in memory with name: no-name-068a0a5a-0c3f-42e9-be25-fa3b0565ae22
Trial: {'body_learning_rate': 2.1490552520467495e-05, 'num_epochs': 4, 'batch_size': 16, 'seed': 4, 'max_iter': 74, 'solver': 'lbfgs'}
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 1706
  Batch size = 16
  Num epochs = 4


Step,Training Loss
1,0.269300
50,0.278500
100,0.282000
150,0.277500
200,0.273900


***** Running evaluation *****
[I 2024-11-27 17:16:10,069] Trial 0 finished with value: 0.5714285714285714 and parameters: {'body_learning_rate': 2.1490552520467495e-05, 'num_epochs': 4, 'batch_size': 16, 'seed': 4, 'max_iter': 74, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.5714285714285714.
Trial: {'body_learning_rate': 6.207321699607185e-06, 'num_epochs': 2, 'batch_size': 8, 'seed': 4, 'max_iter': 130, 'solver': 'liblinear'}
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a down

Step,Training Loss
1,0.350100
50,0.269800
100,0.286400
150,0.289900
200,0.271300


***** Running evaluation *****
[I 2024-11-27 17:17:14,967] Trial 1 finished with value: 0.5714285714285714 and parameters: {'body_learning_rate': 6.207321699607185e-06, 'num_epochs': 2, 'batch_size': 8, 'seed': 4, 'max_iter': 130, 'solver': 'liblinear'}. Best is trial 0 with value: 0.5714285714285714.
Trial: {'body_learning_rate': 0.0002172939771001778, 'num_epochs': 4, 'batch_size': 8, 'seed': 29, 'max_iter': 148, 'solver': 'newton-cg'}
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a 

...

***** Running evaluation *****
[I 2024-11-27 17:42:45,773] Trial 20 finished with value: 0.5714285714285714 and parameters: {'body_learning_rate': 6.05547499639898e-05, 'num_epochs': 3, 'batch_size': 16, 'seed': 12, 'max_iter': 71, 'solver': 'liblinear'}. Best is trial 0 with value: 0.5714285714285714.
Trial: {'body_learning_rate': 0.000993742028997601, 'num_epochs': 4, 'batch_size': 8, 'seed': 31, 'max_iter': 145, 'solver': 'newton-cg'}
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a 

Step,Training Loss
1,0.406800
50,0.282900
100,0.272700
150,0.281900
200,0.274400
250,0.279900
300,0.279700
350,0.270000
400,0.281600


***** Running evaluation *****
[I 2024-11-27 17:44:53,571] Trial 21 finished with value: 0.5714285714285714 and parameters: {'body_learning_rate': 0.000993742028997601, 'num_epochs': 4, 'batch_size': 8, 'seed': 31, 'max_iter': 145, 'solver': 'newton-cg'}. Best is trial 0 with value: 0.5714285714285714.
Trial: {'body_learning_rate': 2.1693099973854055e-05, 'num_epochs': 4, 'batch_size': 8, 'seed': 29, 'max_iter': 155, 'solver': 'newton-cg'}
/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on 

Step,Training Loss
1,0.273300
50,0.274500


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

/home/mun/miniconda3/envs/agreementclassifier/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 1706
  Batch size = 16
  Num epochs = 4


Step,Training Loss
1,0.269300
50,0.278500
100,0.282000
150,0.277500
200,0.273900


In [12]:
trainer.evaluate()

***** Running evaluation *****


{'accuracy': 0.5714285714285714}

In [13]:
trainer.evaluate(ttv_dataset['test'])

***** Running evaluation *****


{'accuracy': 0.75}

In [14]:
trainer.model.save_pretrained("../models/agreement/target_sv")

In [23]:
trainer.model.predict(["Konfigurerbara system ska ha sina konfigurationer lagrade i servicedatorn."])

tensor([0])